# Tekstiklassifikatsiooni ülesanne

Selles moodulis alustame lihtsa tekstiklassifikatsiooni ülesandega, mis põhineb **[AG_NEWS](http://www.di.unipi.it/~gulli/AG_corpus_of_news_articles.html)** andmestikul: klassifitseerime uudiste pealkirjad ühte neljast kategooriast: Maailm, Sport, Äri ja Teadus/Tehnoloogia.

## Andmestik

Andmestiku laadimiseks kasutame **[TensorFlow Datasets](https://www.tensorflow.org/datasets)** API-d.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

# In this tutorial, we will be training a lot of models. In order to use GPU memory cautiously,
# we will set tensorflow option to grow GPU memory allocation when required.
physical_devices = tf.config.list_physical_devices('GPU') 
if len(physical_devices)>0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

dataset = tfds.load('ag_news_subset')

Nüüd saame kasutada andmekogumi treening- ja testimisosi, kasutades vastavalt `dataset['train']` ja `dataset['test']`:


In [3]:
ds_train = dataset['train']
ds_test = dataset['test']

print(f"Length of train dataset = {len(ds_train)}")
print(f"Length of test dataset = {len(ds_test)}")

Length of train dataset = 120000
Length of test dataset = 7600


Prindime välja meie andmekogumi esimesed 10 uut pealkirja:


In [4]:
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

for i,x in zip(range(5),ds_train):
    print(f"{x['label']} ({classes[x['label']]}) -> {x['title']} {x['description']}")

3 (Sci/Tech) -> b'AMD Debuts Dual-Core Opteron Processor' b'AMD #39;s new dual-core Opteron chip is designed mainly for corporate computing applications, including databases, Web services, and financial transactions.'
1 (Sports) -> b"Wood's Suspension Upheld (Reuters)" b'Reuters - Major League Baseball\\Monday announced a decision on the appeal filed by Chicago Cubs\\pitcher Kerry Wood regarding a suspension stemming from an\\incident earlier this season.'
2 (Business) -> b'Bush reform may have blue states seeing red' b'President Bush #39;s  quot;revenue-neutral quot; tax reform needs losers to balance its winners, and people claiming the federal deduction for state and local taxes may be in administration planners #39; sights, news reports say.'
3 (Sci/Tech) -> b"'Halt science decline in schools'" b'Britain will run out of leading scientists unless science education is improved, says Professor Colin Pillinger.'
1 (Sports) -> b'Gerrard leaves practice' b'London, England (Sports Network

## Teksti vektorisatsioon

Nüüd peame teksti teisendama **numbriteks**, mida saab esitada tensoritena. Kui soovime sõnatasemel esindust, peame tegema kahte asja:

* Kasutama **tokeniseerijat**, et jagada tekst **tokeniteks**.
* Looma nende tokenite **sõnavara**.

### Sõnavara suuruse piiramine

AG News andmestiku näites on sõnavara suurus üsna suur, üle 100 000 sõna. Üldiselt ei vaja me sõnu, mis esinevad tekstis harva &mdash; ainult mõnes lauses on need olemas, ja mudel ei õpi neist midagi. Seega on mõistlik piirata sõnavara suurust väiksemaks, andes vektorisatsiooni konstruktorile vastava argumendi:

Mõlemad sammud saab teostada **TextVectorization** kihiga. Loome vektorisatsiooni objekti ja kutsume seejärel `adapt` meetodi, et läbida kogu tekst ja koostada sõnavara:


In [5]:
vocab_size = 50000
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size)
vectorizer.adapt(ds_train.take(500).map(lambda x: x['title']+' '+x['description']))

> **Märkus**: Kasutame ainult osa kogu andmestikust, et koostada sõnavara. Teeme seda, et kiirendada täitmise aega ja vältida teie ootamist. Siiski võtame riski, et mõned sõnad kogu andmestikust ei pruugi sõnavarasse jõuda ja jäävad treeningu ajal tähelepanuta. Seega, kasutades kogu sõnavara suurust ja läbides kogu andmestiku `adapt` käigus, peaks lõplik täpsus suurenema, kuid mitte märkimisväärselt.

Nüüd saame ligi tegelikule sõnavarale:


In [6]:
vocab = vectorizer.get_vocabulary()
vocab_size = len(vocab)
print(vocab[:10])
print(f"Length of vocabulary: {vocab_size}")

['', '[UNK]', 'the', 'to', 'a', 'in', 'of', 'and', 'on', 'for']
Length of vocabulary: 5335


Vektorisaatori abil saame hõlpsasti kodeerida mis tahes teksti numbrite kogumiks:


In [7]:
vectorizer('I love to play with my words')

<tf.Tensor: shape=(7,), dtype=int64, numpy=array([ 112, 3695,    3,  304,   11, 1041,    1], dtype=int64)>

## Sõnade kogumi tekstiesitus

Kuna sõnad kannavad tähendust, saame mõnikord teksti tähenduse välja selgitada, vaadates ainult üksikuid sõnu, sõltumata nende järjekorrast lauses. Näiteks uudiste klassifitseerimisel viitavad sõnad nagu *ilm* ja *lumi* tõenäoliselt *ilmaennustusele*, samas kui sõnad nagu *aktsiad* ja *dollar* viitavad *finantsuudistele*.

**Sõnade kogumi** (BoW) vektor-esitus on kõige lihtsamini mõistetav traditsiooniline vektor-esitus. Iga sõna on seotud vektori indeksiga ning vektori element sisaldab iga sõna esinemiskordade arvu antud dokumendis.

![Pilt, mis näitab, kuidas sõnade kogumi vektor-esitust mälus kujutatakse.](../../../../../translated_images/et/bag-of-words-example.606fc1738f1d7ba9.webp) 

> **Note**: BoW-d võib mõelda ka kui kõigi teksti üksiksõnade ühekuumkooditud vektorite summana.

Allpool on näide, kuidas luua sõnade kogumi esitust, kasutades Scikit Learn Python'i teeki:


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
sc_vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
sc_vectorizer.fit_transform(corpus)
sc_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

Me saame kasutada ka ülal määratletud Kerase vektoreerijat, teisendades iga sõna numbri ühekuumkoodiks ja liites kõik need vektorid kokku:


In [9]:
def to_bow(text):
    return tf.reduce_sum(tf.one_hot(vectorizer(text),vocab_size),axis=0)

to_bow('My dog likes hot dogs on a hot day.').numpy()

array([0., 5., 0., ..., 0., 0., 0.], dtype=float32)

> **Märkus**: Võib-olla üllatab teid, et tulemus erineb eelmisest näitest. Põhjus on selles, et Keras näites vastab vektori pikkus sõnavara suurusele, mis loodi kogu AG News andmestiku põhjal, samas kui Scikit Learn näites ehitasime sõnavara jooksvalt näidisteksti põhjal.


## BoW klassifikaatori treenimine

Nüüd, kui oleme õppinud, kuidas luua tekstist sõnapilve (bag-of-words) esitus, treenime klassifikaatorit, mis seda kasutab. Kõigepealt peame oma andmestiku teisendama sõnapilve esituseks. Seda saab teha, kasutades `map` funktsiooni järgmiselt:


In [11]:
batch_size = 128

ds_train_bow = ds_train.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)
ds_test_bow = ds_test.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)

Nüüd defineerime lihtsa klassifitseeriva närvivõrgu, mis sisaldab ühte lineaarset kihti. Sisendi suurus on `vocab_size` ja väljundi suurus vastab klasside arvule (4). Kuna lahendame klassifitseerimisülesannet, on lõplik aktivatsioonifunktsioon **softmax**:


In [12]:
model = keras.models.Sequential([
    keras.layers.Dense(4,activation='softmax',input_shape=(vocab_size,))
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train_bow,validation_data=ds_test_bow)

938/938 [==============================] - 66s 70ms/step - loss: 0.6144 - acc: 0.8427 - val_loss: 0.4416 - val_acc: 0.8697


Kuna meil on 4 klassi, siis üle 80% täpsus on hea tulemus.

## Klassifikaatori treenimine ühe võrguna

Kuna vektoriseerija on samuti Keras kiht, saame defineerida võrgu, mis sisaldab seda, ja treenida seda algusest lõpuni. Sel viisil ei pea me andmekogumit vektoriseerima, kasutades `map`, vaid saame lihtsalt edastada algse andmekogumi võrgu sisendisse.

> **Note**: Me peame siiski rakendama kaarte oma andmekogumile, et teisendada väljad sõnastikest (nagu `title`, `description` ja `label`) tuplideks. Kuid andmeid kettalt laadides saame kohe alguses luua andmekogumi vajaliku struktuuriga.


In [13]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

inp = keras.Input(shape=(1,),dtype=tf.string)
x = vectorizer(inp)
x = tf.reduce_sum(tf.one_hot(x,vocab_size),axis=1)
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
model.summary()

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 tf.one_hot (TFOpLambda)     (None, None, 5335)        0         
                                                                 
 tf.math.reduce_sum (TFOpLam  (None, 5335)             0         
 bda)                                                            
                                                                 
 dense_2 (Dense)             (None, 4)                 21344     
                                                                 
Total params: 21,344
Trainable params: 21,344
Non-trainable p

## Bigrammid, trigrammid ja n-grammid

Üks sõnakoti lähenemise piirangutest on see, et mõned sõnad on osa mitmesõnalistest väljenditest. Näiteks sõnal 'hot dog' on täiesti erinev tähendus võrreldes sõnadega 'hot' ja 'dog' teistes kontekstides. Kui me esindame sõnu 'hot' ja 'dog' alati samade vektoritega, võib see meie mudelit segadusse ajada.

Selle probleemi lahendamiseks kasutatakse sageli **n-grammide esitusviise** dokumentide klassifitseerimise meetodites, kus iga sõna, kahe- või kolmesõnalise kombinatsiooni sagedus on kasulik tunnus klassifikaatorite treenimiseks. Näiteks bigrammide esituses lisame sõnavarasse kõik sõnapaarid lisaks algsetele sõnadele.

Allpool on näide, kuidas luua bigrammide sõnakoti esitus Scikit Learn'i abil:


In [14]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

N-grammi lähenemise peamine puudus on see, et sõnavara suurus hakkab väga kiiresti kasvama. Praktikas peame kombineerima n-grammi esitusviisi dimensioonide vähendamise tehnikaga, näiteks *embeddings*, mida arutame järgmises osas.

Et kasutada n-grammi esitusviisi meie **AG News** andmestikus, peame `ngrams` parameetri edastama `TextVectorization` konstruktorile. Bigrammi sõnavara pikkus on **märkimisväärselt suurem**, meie puhul üle 1,3 miljoni tokeni! Seetõttu on mõistlik piirata ka bigrammi tokenite arvu mõistliku piirini.

Me võiksime kasutada sama koodi, mis ülalpool, et treenida klassifikaatorit, kuid see oleks väga mälumahukas. Järgmises osas treenime bigrammi klassifikaatorit kasutades embeddings. Seniks saate katsetada bigrammi klassifikaatori treenimist selles märkmikus ja vaadata, kas suudate saavutada kõrgema täpsuse.


## BoW vektorite automaatne arvutamine

Eelmises näites arvutasime BoW vektorid käsitsi, summeerides üksiksõnade ühekuumkoodid. Kuid TensorFlow uusim versioon võimaldab meil BoW vektoreid automaatselt arvutada, kui edastame vektoriseerija konstruktorile parameetri `output_mode='count'`. See muudab meie mudeli defineerimise ja treenimise märkimisväärselt lihtsamaks:


In [15]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='count'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 7s 7ms/step - loss: 0.5929 - acc: 0.8486 - val_loss: 0.4168 - val_acc: 0.8772


## Termi sagedus - pöörddokumendi sagedus (TF-IDF)

BoW-esituses kaalutakse sõnade esinemist sama tehnikaga, sõltumata sõnast endast. Siiski on selge, et sagedased sõnad nagu *a* ja *in* on klassifitseerimise jaoks palju vähem olulised kui spetsialiseeritud terminid. Enamikus NLP-ülesannetes on mõned sõnad olulisemad kui teised.

**TF-IDF** tähistab **termi sagedust - pöörddokumendi sagedust**. See on bag-of-words'i variatsioon, kus binaarse 0/1 väärtuse asemel, mis näitab sõna esinemist dokumendis, kasutatakse ujuvpunkti väärtust, mis on seotud sõna esinemissagedusega korpuses.

Formaalsemalt on sõna $i$ kaal dokumendis $j$, tähistatud kui $w_{ij}$, defineeritud järgmiselt:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
kus
* $tf_{ij}$ on sõna $i$ esinemiste arv dokumendis $j$, st BoW väärtus, mida oleme varem näinud
* $N$ on dokumentide arv kogumis
* $df_i$ on dokumentide arv, mis sisaldavad sõna $i$ kogu kogumis

TF-IDF väärtus $w_{ij}$ suureneb proportsionaalselt sõna esinemiste arvuga dokumendis ja seda korrigeeritakse dokumentide arvu järgi korpuses, mis sisaldavad seda sõna. See aitab arvestada asjaolu, et mõned sõnad esinevad sagedamini kui teised. Näiteks, kui sõna esineb *iga* dokumendis kogumis, siis $df_i=N$ ja $w_{ij}=0$, ning need terminid jäetakse täielikult kõrvale.

TF-IDF vektorisatsiooni teksti jaoks saab hõlpsasti luua Scikit Learn'i abil:


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

Kerases saab `TextVectorization` kiht automaatselt arvutada TF-IDF sagedusi, kui edastada parameeter `output_mode='tf-idf'`. Kordame ülaltoodud koodi, et näha, kas TF-IDF kasutamine suurendab täpsust:


In [17]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='tf-idf'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 12s 12ms/step - loss: 0.4197 - acc: 0.8662 - val_loss: 0.3432 - val_acc: 0.8849


## Kokkuvõte

Kuigi TF-IDF esitlused annavad erinevatele sõnadele sageduskaalud, ei suuda need esitada tähendust ega järjekorda. Nagu kuulus lingvist J. R. Firth ütles 1935. aastal: "Sõna täielik tähendus on alati kontekstuaalne ja ükski tähenduse uurimine väljaspool konteksti ei saa olla tõsiseltvõetav." Kursuse käigus õpime hiljem, kuidas tekstist kontekstuaalset teavet keelemudelite abil haarata.



---

**Lahtiütlus**:  
See dokument on tõlgitud AI tõlketeenuse [Co-op Translator](https://github.com/Azure/co-op-translator) abil. Kuigi püüame tagada täpsust, palume arvestada, et automaatsed tõlked võivad sisaldada vigu või ebatäpsusi. Algne dokument selle algses keeles tuleks pidada autoriteetseks allikaks. Olulise teabe puhul soovitame kasutada professionaalset inimtõlget. Me ei vastuta selle tõlke kasutamisest tulenevate arusaamatuste või valesti tõlgenduste eest.
